# Hyperparameters finder for xGBoost

In [ ]:
# Imports & dataset opening
from datetime import datetime

import pandas as pd

from Models.Pairs.Hyperparameters.hyperopt_hyperparameters import HyperOptXgboost
from Models.Pairs.Hyperparameters.optuna_hyperparameters import OptunaXgboost
from Models.variables import PAIRS_DATASET, HYPEROPT_EVALUATIONS, OPTUNA_TRIALS

dataset = pd.read_parquet(f"../../../Dataset/Datasets/{PAIRS_DATASET}")
number_of_trials = 50
# method = 'Hyperopt'
method = 'Optuna'

In [ ]:
# Run solution finder
results = list()

if method == 'Hyperopt':
    for i in range(number_of_trials):
        hyper = HyperOptXgboost(dataset, prints=False, no_of_evals=HYPEROPT_EVALUATIONS)
        results.append([i + 1, hyper.pairs, hyper.accuracy, hyper.accuracy_with_bonuses, hyper.hyperopt_results])
        print(results[-1][:-1])

elif method == 'Optuna':
    for i in range(number_of_trials):
        hyper = OptunaXgboost(dataset, prints=False, no_of_trials=OPTUNA_TRIALS, accuracy_threshold=0.709)
        results.append([i + 1, hyper.pairs, hyper.accuracy, hyper.accuracy_with_bonuses, hyper.optuna_results])
        print(results[-1][:-1])

In [ ]:
# Save results
time_now = datetime.now()

filename = f"Solutions/{method}/solution_{time_now.strftime('%m%d')}_{time_now.strftime('%H%M')}_{PAIRS_DATASET}"

df = pd.DataFrame(data=results, columns=['Run', 'Pairs_accuracy', 'Accuracy', 'Accuracy_with_bonuses', f'{method}_results'])
df.to_parquet(filename)

In [ ]:
# Open results
opened = pd.read_parquet(filename)
opened = opened.sort_values(['Accuracy_with_bonuses', 'Accuracy', 'Pairs_accuracy'], ascending=[False, False, False]).reset_index(drop=True)
opened

In [ ]:
opened.loc[0, f'{method}_results']